In [9]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
# transformador para selecionar características baseadas em pesos de importância
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import MinMaxScaler

In [10]:
df = pd.read_csv("./kddcup99-dataset/kddcup99.csv")

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494020 entries, 0 to 494019
Data columns (total 42 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     494020 non-null  int64  
 1   protocol_type                494020 non-null  object 
 2   service                      494020 non-null  object 
 3   flag                         494020 non-null  object 
 4   src_bytes                    494020 non-null  int64  
 5   dst_bytes                    494020 non-null  int64  
 6   land                         494020 non-null  int64  
 7   wrong_fragment               494020 non-null  int64  
 8   urgent                       494020 non-null  int64  
 9   hot                          494020 non-null  int64  
 10  num_failed_logins            494020 non-null  int64  
 11  logged_in                    494020 non-null  int64  
 12  lnum_compromised             494020 non-null  int64  
 13 

In [12]:
le = LabelEncoder()
df['lbl'] = df['label'].copy()
df['lbl'] = le.fit_transform(df['lbl'])
df[['label', 'lbl']].drop_duplicates()

,label,lbl
0,normal,11
744,buffer_overflow,1
4049,loadmodule,7
4113,perl,12
7601,neptune,9
7793,smurf,18
15699,guess_passwd,3
15784,pod,14
19286,teardrop,20
22814,portsweep,15


In [13]:
def geraLabels(df, lista):
    le = LabelEncoder()
    for i in lista:
        df[i] = le.fit_transform(df[i])
    return df

In [14]:
# numeric_cols

In [15]:
colunasCategoricas = ['label', 'protocol_type', 'service', 'flag']
colunasInput = ['protocol_type', 'service', 'flag']

df = geraLabels(df, colunasCategoricas)
scaler = MinMaxScaler()

# Calcula o mínimo e o máximo para ser usado pra escalar os valores
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()[:-1]
scaler.fit(df[numeric_cols])
df[numeric_cols] = scaler.transform(df[numeric_cols])

In [16]:
train_df, test_df = train_test_split(df, test_size=0.2)
print(train_df.shape, test_df.shape)

(395216, 43) (98804, 43)


In [ ]:
train_inputs = train_df[input_cols].copy()
train_targets = train_df[target_col].copy()
test_inputs = test_df[input_cols].copy()
test_targets = test_df[target_col].copy()

In [18]:
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100, random_state=42))

# da fit no meta-transformador SelectFromModel
sel.fit(train_df, train_df['label'])
#sel.predict(test_df[selected_feat])
variaveis = train_df.columns[sel.get_support()]

ValueError: Unknown label type: 'continuous'

In [ ]:
variaveis = ['protocol_type', 'service', 'flag', 'dst_bytes', 'count', 'srv_count',
       'same_srv_rate', 'diff_srv_rate', 'dst_host_srv_count',
       'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
       'dst_host_same_src_port_rate']

In [ ]:
rf = RandomForestClassifier(n_estimators = 2000, random_state = 42)
# realiza o fit passando os inputs que irão receber o treinamento, com as colunas selecionadas no passo anterior e os alvos
rf.fit(train_df[variaveis], train_df['label']);

# realiza a predição das classes com os inputs de test com as colunas selecionadas pelo feat
preds_rf = rf.predict(test_df[variaveis])

In [ ]:
from sklearn.metrics import accuracy_score

# com base nas classes observadas anteriormente, é verificado quais delas se comportam com o esperado
score_rf = accuracy_score(test_df['label'], preds_rf)

In [ ]:
score_rf * 100

99.88664426541436

In [ ]:
preds_rf

array([18, 18, 11, ...,  9, 18,  9])